In [ ]:
from gurobipy import * 
import utils_anna
import render_anna # For presenting layouts for rendering in Jupyter 
from IPython.display import SVG # SVG capabilities for showing layouts
%matplotlib inline

### This is the solution to the exercises from the <a href="exact methods.ipynb"> exact methods </a> notbeook

## Exercise 2: Trading off different user groups
We modify the linear assignment problem for menus to take into account both frequencies as follows: 

$$min \sum_{i=1}^N\sum_{j=1}^N \color{red}{(0.5 \cdot p^{novice}_i + 0.5 \cdot p^{expert}_i)} \cdot d_j \cdot r \cdot x_{ij} $$
$$\text{subject to} \hspace{6cm} $$
$$\sum_{i=1}^N x_{ij} = 1\hspace{1cm} \forall j = 1 .. N$$
$$\sum_{j=1}^N x_{ij} = 1\hspace{1cm} \forall i = 1 .. N$$
$$x_{ij} \in {0, 1} \hspace{1.6cm} \forall i, j = 1 .. N$$

The frequencies are weighted by 0.5 to preserve the range of the objective value. Similarly, different weights could be used to favor different user groups.

## Exercise 3a: Formulation of the letter assignment problem
<img src="imgs/letter_assignment2.PNG" style="float:right" width=40%>

We can use the same formulation as for the quadratic assignmenr problem where we have assigned factories to loactions. The flow between factories corresponds to the frequency of typing one letter after another. The distance between locations corresponds to the time it takes to move bettween two keys on the keyboard. For typing with one finger, this can be quanitifed by Fitts' Law. The goal is to assign characters to keys such that the time to type any character after any other character is minimized: 

$$ min \sum_{i\in \Sigma}\sum_{k\in K}\sum_{j \in \Sigma} \sum_{l\in k}
p_{ij} \cdot t_{kl} \cdot x_{ik} \cdot x_{jl} $$
$$s.t.$$
$$ \sum_{k \in G} x_{ik} = 1 \qquad \forall i \in \Sigma$$
$$ \sum_{i \in \Sigma} x_{ik} \leq 1 \qquad \forall k \in K$$
$$ x_{ik} \in {0,1} \quad \forall i \in \Sigma, k \in K $$



## Exercise 3b: Implementation of the letter assignment problem
Note: we have only optimized a keyboard layout with 9 letters and keys. Why? Because the time to solve this problem grows exponentially with the number of letters and keys. Here is the runtime on a standard laptop for 9, 10, and 11 keys. If you want you can increase the number of letters and let it run over night. Even if it does not finish, Gurobi reports the objective value and the gap to the global optimum. You can also add a callback function as a parameter to the <code>optimize()</code> function. In the callback function you can write out incumbent solutions to look at them later. See an example <a href="http://www.gurobi.com/documentation/7.0/examples/callback_py.html">here</a>

|number of characters | run time|
|----------|------|
| 9 | 4.5 s|
|10 | 175 s|
|11 | 1.5 h|


In [ ]:
# SOLUTION
import numpy as np
def solve(characters, keyslots, bigram_frequency, movement_time, columns):
    # ==== 1. Create the (empty) model ====
    model = Model("keyboard")

    # ==== 2. Add decision variables ======
    x = {}
    # Create one binary variable for each letter-key pair. 
    # We give it a meaningful name so we later understand what it means if it is set to 1
    for i in characters:        
        for k in keyslots:                
            x[(i,k)] = model.addVar(vtype=GRB.BINARY, name="%s_%i"%(i,k))            
    # Integrate new variables
    model.update()

    # ==== 3. Specify Objective function ======        
    cost = quicksum(bigram_frequency[i,j] * movement_time[k,l] * x[(i,k)] * x[(j,l)]  
                                for l in keyslots
                                   for k in keyslots
                                       for i in characters
                                           for j in characters)
    model.setObjective(cost,GRB.MINIMIZE)

    # ====4. Add Constraints ======
    # Add constraints
    # Each letter is only assigned to one keyslot
    for i in characters: 
        model.addConstr(quicksum(x[(i,k)]
                           for k in keyslots) == 1, "uniqueness_constraint_%s"%i)    
    # Each element is only assigned to one position
    for k in keyslots: 
        model.addConstr(quicksum(x[(i,k)]
                           for i in characters) <= 1, "uniqueness_constraint_%i"%k)
        
    #Bonus task: HCI constraint
    if "h" in characters and "c" in characters and "i" in characters:
        #distance between h and c is 1
        model.addConstr(quicksum(x[("h",k)] * x[("c",l)] * utils_anna.distance(columns, k, l)
                                 for k in keyslots 
                                     for l in keyslots) == 1 )
        # and the horizontal distance is 1
        model.addConstr(quicksum(x[("h",k)] * x[("c",l)] * (l-k)
                                for k in keyslots 
                                     for l in keyslots) == 1 )
        
        #distance between c and i is 1
        model.addConstr(quicksum(x[("c",k)] * x[("i",l)] * utils_anna.distance(columns, k, l)
                                 for k in keyslots 
                                     for l in keyslots) == 1 )
        # and the horizontal distance is 1
        model.addConstr(quicksum(x[("c",k)] * x[("i",l)] * (l-k)
                                 for k in keyslots 
                                     for l in keyslots) == 1 )
    
    model.update()
    
    # ==== 5. Optimize model ======    
    p=model.presolve()
    p.write("presolve.lp")
    model.optimize()
    
    # ====6. Extract solution ======   
    mapping = {}
    
    for v in model.getVars():
        if v.x == 1:
            character = v.varName.split("_")[0]
            slot = int(v.varName.split("_")[1])
            mapping[character] = slot
            
    return mapping, model.getObjective().getValue()

In [ ]:
#define characters and keyslots
characters = ['a', 'c', 's', 'r', 'e', 'f', 't', 'h', 'i']
keyslots = list(range(len(characters))) 
columns = 3

#obtain cost factors: movement time and bigram frequencies
movement_time = {(s1,s2): utils_anna.fittslawcost(s1,s2, utils_anna.distance(columns,s1,s2)) for s1 in keyslots for s2 in keyslots}

#letter pair frequency        
bigram_frequency = utils_anna.get_bigram_frequency(characters)

#solve the problem
mapping, objective = solve(characters, keyslots, bigram_frequency, movement_time, columns)

print "The average WPM of the winning keyboard is %.2f"%utils_anna.wpm(objective)
render_anna.plot_keyboard(mapping, columns)

## Bonus task: 
The distance between the keys mapped to H-C and C-I is 1: 
$$ \sum_{k\in K} \sum_{l\in K} d_{kl} \cdot x_{\mathbf{ H }k} \cdot x_{\mathbf{ C }l} = 1$$
$$ \sum_{k\in K} \sum_{l\in K} d_{kl} \cdot x_{\mathbf{ C }k} \cdot x_{\mathbf{ I }l} = 1$$

The key mapped to H is left of the key mapped to C, same for C and I:
$$ \sum_{k\in K} \sum_{l\in K} (l-k) \cdot x_{\mathbf{ H }k} \cdot x_{\mathbf{ C }l} = 1$$
$$ \sum_{k\in K} \sum_{l\in K} (l-k) \cdot x_{\mathbf{ C }k} \cdot x_{\mathbf{ I }l} = 1$$

The implementation is shown in the <code>solve()</code> function above. 

Note: while constraints may make the problem easier to solve because they reduce the space of feasible solutions, quadratic constraints like these further increase the mathematical complexity of the problem. 